# Scrape AWS Products, Categories and Icons

In [1]:
from urllib.parse import parse_qs

from bs4 import BeautifulSoup
import pandas as pd
import requests

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
AWS_BASE_URL = 'https://aws.amazon.com'
soup = BeautifulSoup(requests.get(f'{AWS_BASE_URL}/products').text, 'html.parser')

In [3]:
category_soups = soup.find_all('a', {'class': 'lb-trigger'})
category_soups[:1]

[<a class="lb-trigger lb-icon-ps-60-pie-bar-charts"> <span>Analytics</span> </a>]

In [4]:
aws_categories = []
for i, cat in enumerate(category_soups):
    aws_categories.append({
        'category_id': i + 1,
        'category_name': cat.text.strip()
    })
aws_categories_df = pd.DataFrame(aws_categories)
aws_categories_df.head()

,category_id,category_name
0,1,Analytics
1,2,Application Integration
2,3,AR & VR
3,4,AWS Cost Management
4,5,Blockchain


In [5]:
service_soups = [s.find('a') for s in soup.find_all('div', {'class': 'lb-content-item'})]
service_soups[:1]

[<a href="/athena/?c=1&amp;pt=1"> Amazon Athena<span>Query Data in S3 using SQL</span> </a>]

In [9]:
aws_services = []

for s in service_soups:
    url = f"{AWS_BASE_URL}{s['href']}"
    text = s.text.strip()
    desc = s.find('span').text.strip()
    name = text[:text.index(desc)]
    
    service_detail_soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    full_description = '\n'.join([p.text for p in service_detail_soup.find_all('p')[:3]])
    
    aws_services.append({
        'category_id': int(parse_qs(url.split('?')[1])['c'][0]),
        'name': name,
        'short_description': desc,
        'long_description': full_description,
        'link': url
    })
aws_services_df = pd.DataFrame(aws_services)
aws_services_df.head()

,category_id,link,long_description,name,short_description
0,1,https://aws.amazon.com/athena/?c=1&pt=1,Amazon Athena is an interactive query service ...,Amazon Athena,Query Data in S3 using SQL
1,1,https://aws.amazon.com/cloudsearch/?c=1&pt=2,Related Links\nGet Started for Free\nAmazon Cl...,Amazon CloudSearch,Managed Search Service
2,1,https://aws.amazon.com/elasticsearch-service/?...,Amazon Elasticsearch Service is a fully manage...,Amazon Elasticsearch Service,Run and Scale Elasticsearch Clusters
3,1,https://aws.amazon.com/elasticmapreduce/?c=1&pt=4,Amazon EMR provides a managed Hadoop framework...,Amazon EMR,Hosted Hadoop Framework
4,1,https://aws.amazon.com/kinesis/?c=1&pt=5,"Amazon Kinesis makes it easy to collect, proce...",Amazon Kinesis,Work with Real-time Streaming Data


In [10]:
aws_df = pd.merge(aws_categories_df, aws_services_df, on='category_id')
aws_df.head()

,category_id,category_name,link,long_description,name,short_description
0,1,Analytics,https://aws.amazon.com/athena/?c=1&pt=1,Amazon Athena is an interactive query service ...,Amazon Athena,Query Data in S3 using SQL
1,1,Analytics,https://aws.amazon.com/cloudsearch/?c=1&pt=2,Related Links\nGet Started for Free\nAmazon Cl...,Amazon CloudSearch,Managed Search Service
2,1,Analytics,https://aws.amazon.com/elasticsearch-service/?...,Amazon Elasticsearch Service is a fully manage...,Amazon Elasticsearch Service,Run and Scale Elasticsearch Clusters
3,1,Analytics,https://aws.amazon.com/elasticmapreduce/?c=1&pt=4,Amazon EMR provides a managed Hadoop framework...,Amazon EMR,Hosted Hadoop Framework
4,1,Analytics,https://aws.amazon.com/kinesis/?c=1&pt=5,"Amazon Kinesis makes it easy to collect, proce...",Amazon Kinesis,Work with Real-time Streaming Data


In [ ]:
def 